In [19]:
import random
from collections import Counter

Voici un rappel de la fonction si on veut en générer d'autres :

In [26]:
def generate_preferences_full(num_people):
    men_preferences = {}
    women_preferences = {}

    all_people = [f"m{i}" for i in range(1, num_people + 1)] + [f"w{i}" for i in range(1, num_people + 1)]

    for person in all_people:
        if person.startswith("m"):
            preferences = [woman for woman in all_people if woman.startswith("w") and woman != person]
            random.shuffle(preferences)
            men_preferences[person] = preferences
        elif person.startswith("w"):
            preferences = [man for man in all_people if man.startswith("m") and man != person]
            random.shuffle(preferences)
            women_preferences[person] = preferences

    return men_preferences, women_preferences

# Usage
num_people = 5
men_preferences, women_preferences = generate_preferences_full(num_people)
print("Men's preferences:", men_preferences)
print("Women's preferences:", women_preferences)

Men's preferences: {'m1': ['w2', 'w1', 'w4', 'w3', 'w5'], 'm2': ['w1', 'w4', 'w3', 'w2', 'w5'], 'm3': ['w4', 'w2', 'w3', 'w1', 'w5'], 'm4': ['w5', 'w4', 'w3', 'w1', 'w2'], 'm5': ['w1', 'w2', 'w4', 'w5', 'w3']}
Women's preferences: {'w1': ['m5', 'm2', 'm4', 'm3', 'm1'], 'w2': ['m1', 'm5', 'm2', 'm4', 'm3'], 'w3': ['m3', 'm5', 'm1', 'm2', 'm4'], 'w4': ['m4', 'm3', 'm1', 'm5', 'm2'], 'w5': ['m5', 'm4', 'm2', 'm3', 'm1']}


Bonsoir à tous, je propose de partir avec ce beau tableau généré soigneusement avec preferences_set_generation

In [27]:
men_preferences = {'m1': ['w2', 'w1', 'w4', 'w3', 'w5'], 'm2': ['w1', 'w4', 'w3', 'w2', 'w5'], 'm3': ['w4', 'w2', 'w3', 'w1', 'w5'], 'm4': ['w5', 'w4', 'w3', 'w1', 'w2'], 'm5': ['w1', 'w2', 'w4', 'w5', 'w3']}
women_preferences = {'w1': ['m5', 'm2', 'm4', 'm3', 'm1'], 'w2': ['m1', 'm5', 'm2', 'm4', 'm3'], 'w3': ['m3', 'm5', 'm1', 'm2', 'm4'], 'w4': ['m4', 'm3', 'm1', 'm5', 'm2'], 'w5': ['m5', 'm4', 'm2', 'm3', 'm1']}


Avant de faire tourner l'algo automatiquement, voilà ce qu'on devrait trouver si on considère qu'on trie en premier les premiers arrivés

m1 -> w4

m2 -> w2

m3 -> w3

m4 -> w5

m5 -> w1

Algo qui marche et qui répond au problème : [Algo Gale-Sharpley en Python](https://towardsdatascience.com/gale-shapley-algorithm-simply-explained-caa344e643c2)

In [29]:
def gale_shapley(women_preferences, men_preferences):# Initialize variables
    waiting_list = []
    proposals = {}
    count = 0

    # dict to control which women each man can make proposals
    women_available = {man: list(women_preferences.keys()) for man in men_preferences.keys()}
    men_available = men_preferences.copy()

    # while not all men have pairs
    while len(waiting_list) < len(men_preferences):
        # man makes proposals
        for man in men_preferences.keys():
            if man not in waiting_list:
                # each man make proposal to the top women from his list
                women = men_available[man]
                best_choice = women[0]  # Man proposes to his top preference woman
                proposals[(man, best_choice)] = (women_preferences[best_choice].index(man), men_preferences[man].index(best_choice))
                del women_available[man][0]  # Remove the proposed woman from man's available choices
        # if women have more than one proposals
        # she will choose the best option
        overlays = Counter([key[1] for key in proposals.keys()])
        # cycle to choose the best options
        for woman in overlays.keys():
            if overlays[woman] > 1:
                # pairs to drop from proposals
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys()
                                        if woman in pair}.items(),
                                    key=lambda x: (x[1][1], x[1][0]))[1:]
                # if man was rejected by woman
                # there is no point for him to make proposal
                # second time to the same woman
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    del men_available[p_to_drop[0][0]][0]  # Remove the proposed woman from man's available choices
        # man who successfully created pairs must be added to the waiting list
        waiting_list = [man[0] for man in proposals.keys()]
        # update counter
        count += 1

    out = "Stable Matching Pairs:"

    for pair in proposals:
        out += f'\n{pair[0]} <---> {pair[1]}'

    return out


In [30]:
print(gale_shapley(women_preferences, men_preferences))

Stable Matching Pairs:
m1 <---> w2
m3 <---> w4
m4 <---> w5
m5 <---> w1
m2 <---> w3


In [31]:
%timeit gale_shapley(women_preferences, men_preferences)

3.71 µs ± 8.46 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Nouveau test : que faire s'il y a plus de femmes que d'hommes ?

In [33]:
def generate_preferences_full(num_men, num_women):
    men_preferences = {}
    women_preferences = {}

    men = [f"m{i}" for i in range(1, num_men + 1)]
    women = [f"w{i}" for i in range(1, num_women + 1)]
    all_people = men + women

    for person in all_people:
        if person.startswith("m"):
            preferences = [woman for woman in women]
            random.shuffle(preferences)
            men_preferences[person] = preferences
        elif person.startswith("w"):
            preferences = [man for man in men]
            random.shuffle(preferences)
            women_preferences[person] = preferences

    return men_preferences, women_preferences

# Usage
num_men = 5
num_women = 8
men_preferences, women_preferences = generate_preferences_full(num_men, num_women)
print("Men's preferences:", men_preferences)
print("Women's preferences:", women_preferences)

Men's preferences: {'m1': ['w1', 'w7', 'w5', 'w4', 'w6', 'w2', 'w8', 'w3'], 'm2': ['w4', 'w8', 'w2', 'w3', 'w5', 'w6', 'w1', 'w7'], 'm3': ['w2', 'w8', 'w4', 'w3', 'w7', 'w5', 'w6', 'w1'], 'm4': ['w7', 'w4', 'w6', 'w2', 'w1', 'w5', 'w8', 'w3'], 'm5': ['w2', 'w6', 'w5', 'w3', 'w1', 'w7', 'w8', 'w4']}
Women's preferences: {'w1': ['m4', 'm2', 'm5', 'm3', 'm1'], 'w2': ['m4', 'm3', 'm2', 'm1', 'm5'], 'w3': ['m2', 'm5', 'm4', 'm3', 'm1'], 'w4': ['m2', 'm5', 'm1', 'm3', 'm4'], 'w5': ['m5', 'm4', 'm1', 'm2', 'm3'], 'w6': ['m2', 'm1', 'm4', 'm3', 'm5'], 'w7': ['m5', 'm1', 'm2', 'm4', 'm3'], 'w8': ['m5', 'm1', 'm4', 'm3', 'm2']}


In [1]:
men_preferences = {'m1': ['w1', 'w7', 'w5', 'w4', 'w6', 'w2', 'w8', 'w3'], 'm2': ['w4', 'w8', 'w2', 'w3', 'w5', 'w6', 'w1', 'w7'], 'm3': ['w2', 'w8', 'w4', 'w3', 'w7', 'w5', 'w6', 'w1'], 'm4': ['w7', 'w4', 'w6', 'w2', 'w1', 'w5', 'w8', 'w3'], 'm5': ['w2', 'w6', 'w5', 'w3', 'w1', 'w7', 'w8', 'w4']}
women_preferences = {'w1': ['m4', 'm2', 'm5', 'm3', 'm1'], 'w2': ['m4', 'm3', 'm2', 'm1', 'm5'], 'w3': ['m2', 'm5', 'm4', 'm3', 'm1'], 'w4': ['m2', 'm5', 'm1', 'm3', 'm4'], 'w5': ['m5', 'm4', 'm1', 'm2', 'm3'], 'w6': ['m2', 'm1', 'm4', 'm3', 'm5'], 'w7': ['m5', 'm1', 'm2', 'm4', 'm3'], 'w8': ['m5', 'm1', 'm4', 'm3', 'm2']}


In [35]:
def gale_shapley(women_preferences, men_preferences):
    # Initialize variables
    waiting_list = []
    proposals = {}
    count = 0

    # dict to control which women each man can make proposals
    women_available = {man: list(women_preferences.keys()) for man in men_preferences.keys()}
    men_available = men_preferences.copy()

    # while not all men have pairs
    while len(waiting_list) < len(men_preferences):
        # man makes proposals
        for man in men_preferences.keys():
            if man not in waiting_list:
                # each man make proposal to the top women from his list
                women = men_available[man]
                best_choice = women[0]  # Man proposes to his top preference woman
                proposals[(man, best_choice)] = (women_preferences[best_choice].index(man), men_preferences[man].index(best_choice))
                del women_available[man][0]  # Remove the proposed woman from man's available choices
        # if women have more than one proposals
        # she will choose the best option
        overlays = Counter([key[1] for key in proposals.keys()])
        # cycle to choose the best options
        for woman in overlays.keys():
            if overlays[woman] > 1:
                # pairs to drop from proposals
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys()
                                        if woman in pair}.items(),
                                    key=lambda x: (x[1][1], x[1][0]))[1:]
                # if man was rejected by woman
                # there is no point for him to make proposal
                # second time to the same woman
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    del men_available[p_to_drop[0][0]][0]  # Remove the proposed woman from man's available choices
        # man who successfully created pairs must be added to the waiting list
        waiting_list = [man[0] for man in proposals.keys()]
        # update counter
        count += 1

    out = "Stable Matching Pairs:"

    for pair in proposals:
        out += f'\n{pair[0]} <---> {pair[1]}'

    return out


In [36]:
print(gale_shapley(women_preferences, men_preferences))

Stable Matching Pairs:
m1 <---> w1
m2 <---> w4
m3 <---> w2
m4 <---> w7
m5 <---> w6


Exemple d'appel à la route API gale_shapley

In [3]:
import requests
import json

# Combine men and women preferences into a single dictionary
data = {
    "men_preferences": men_preferences,
    "women_preferences": women_preferences
}

# Convert data to JSON string
json_data = json.dumps(data)

# URL of your Flask API endpoint
url = "http://127.0.0.1:5000/api/gale_shapley"

# Send POST request with Content-Type header set to application/json
try:
    response = requests.post(url, data=json_data, headers={"Content-Type": "application/json"})
    response.raise_for_status()  # Raise an exception for HTTP errors
    print(response.json())
except requests.HTTPError as e:
    print(f"HTTP error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

{'result': {'matched_pairs': [{'man': 'm1', 'woman': 'w1'}, {'man': 'm2', 'woman': 'w4'}, {'man': 'm3', 'woman': 'w2'}, {'man': 'm4', 'woman': 'w7'}, {'man': 'm5', 'woman': 'w6'}]}}
